### Данный ноутбук нужно запускать только при поднятом детекторе (на сервере или локально)

In [1]:
import re
import asyncio
import random
import requests
from playwright.async_api import async_playwright as pl

In [2]:
scroll_down = '''
                                const scrollStep = 200; // Размер шага прокрутки (в пикселях)
                                const scrollInterval = 100; // Интервал между шагами (в миллисекундах)

                                const scrollHeight = document.documentElement.scrollHeight;
                                let currentPosition = 0;
                                const interval = setInterval(() => {
                                    window.scrollBy(0, scrollStep);
                                    currentPosition += scrollStep;

                                    if (currentPosition >= scrollHeight) {
                                        clearInterval(interval);
                                    }
                                }, scrollInterval);
                            '''

links = dict()

In [3]:
async def parse_html(page):
    """Find all links in raw HTML by regular expression (re)."""

    a_handle = await page.evaluate_handle("document.body")
    result_handle = await page.evaluate_handle("body => body.innerHTML", a_handle)
    html = str(await result_handle.json_value())
    await result_handle.dispose()

    find = "https://ir.ozone.ru/s3/rp-photo-[0-9]/wc[0-9]00/.*?jpeg"
    vid = "https://ir.ozone.ru/s3/video-.*?cover.jpg"
    pics = re.findall(find, html)
    thumbnails = re.findall(vid, html)
    
    for el in pics:
        hashed_pic = el[40:-5]
        if links.get(hashed_pic):
            continue
        else:
            links[hashed_pic] = el
    
    for el in thumbnails:
        hashed_thumb = el[32:-22]
        if links.get(hashed_thumb):
            continue
        else:
            links[hashed_thumb] = el

In [4]:
def clean_links(links: dict):
    wc = [f"/wc{i}00/" for i in range(1, 10)]
    for resolution in wc:
        for link in links:
            address = links[link]
            if resolution in address:
                links[link] = address.replace(resolution, "/wc1200/")
    for el in links.copy():
        if "/wc1200/" not in links[el]:
            links.pop(el)
    return links

## Введите свой тг айди и артикул товара на Озоне
Свой тг айди можно узнать здесь: https://t.me/getmyid_bot

In [16]:
ARTICLE = "артикул товара"
USER_ID = "твой айди"
DETECTOR_URL = "http://localhost:9090/model/detect" #поменять на `localhost` на адресс сервера, если он поднят на сервере

In [9]:
pl_ctx = await pl().start()

browser = await pl_ctx.firefox.launch(headless=True)
context = await browser.new_context()
page = await context.new_page()

url = "https://www.ozon.ru/product/" + str(ARTICLE)

await page.goto(url, wait_until="load")

<Response url='https://www.ozon.ru/product/711700590' request=<Request url='https://www.ozon.ru/product/711700590' method='GET'>>

In [10]:
try:
    await page.evaluate(scroll_down)
    await page.locator('div[style*="grid-template-columns: repeat(9, minmax(56px, 90px));"]').click(delay=1)
    
    print("Click on locator")
    await asyncio.sleep(1)
except:
    await page.reload()
    await asyncio.sleep(1.5)
    
    await page.evaluate(scroll_down)
    await page.locator('div[style*="grid-template-columns: repeat(9, minmax(56px, 90px));"]').click(delay=1)
    
    print("Click on locator")
    await asyncio.sleep(1)

Click on locator


In [11]:
counter = 0
links_lenght = 0

while True:
    links_lenght = len(links)
    await parse_html(page=page)
    await page.keyboard.press("PageDown", delay=0.8)
    print(f"Earn {len(links)+1} links")
    sleep = random.uniform(1.5, 3)
    await asyncio.sleep(sleep)
    if links_lenght == len(links):
        counter +=1
    else:
        counter = 0
    if counter > 3:
        break

Earn 32 links
Earn 32 links
Earn 39 links
Earn 47 links
Earn 57 links
Earn 68 links
Earn 73 links
Earn 85 links
Earn 97 links
Earn 107 links
Earn 117 links
Earn 127 links
Earn 138 links
Earn 149 links
Earn 160 links
Earn 171 links
Earn 183 links
Earn 192 links
Earn 200 links
Earn 210 links
Earn 221 links
Earn 229 links
Earn 238 links
Earn 241 links
Earn 241 links
Earn 241 links
Earn 241 links
Earn 241 links


### Если количество собранных ссылок сильно меньше количества обзоров на странице товара, то попробуйте перезапустить ячейку выше

In [12]:
await page.close()
await pl_ctx.stop()

In [13]:
clean = clean_links(links)

In [14]:
len(clean)

234

In [15]:
requests.post(DETECTOR_URL, json={"user_id": USER_ID, "images": list(clean.values())})

<Response [200]>

### Можно не дожидаться выполнения верхней ячейки, а перейти в телеграм в диалог с ботом